# Python / Polars (portfolio calculation FT01.FT03)

## libs

In [65]:
import polars as pl
from datetime import datetime
from datetime import date
import pyarrow as pa
import pyarrow.dataset
import random
import string

## user-defined functions

### calculate tescc

In [66]:
#code

## read files

### ft01

In [67]:
#read parquet files ft01
ft01_or425_2016_df = pl.read_parquet("./files/input/remaining/ft01/*")
#display(ft01_or425_2016_df.head())

In [68]:
#separate by year
ft01_or573_2021_df = ft01_or425_2016_df.filter((pl.col("ANIO_VIGENCIA") > 2021))
#ft01_or573_2021_df.select(pl.count())

#set uvt
ft01_or573_2021_df = ft01_or573_2021_df.with_columns(pl
                                                     .when(pl.col("ANIO_VIGENCIA") > 2022)
                                                     .then(42412)
                                                     .otherwise(38004)
                                                     .alias("UVT_SMMLV")) #uvt +2021



#TESCC post 2021
ft01_or573_2021_df = ft01_or573_2021_df.with_columns(pl
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "INDUSTRIAL")
                                                     .then(
                                                         pl
                                                         .when((pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO") * pl.lit(0.017)) > (pl.lit(42.54) * pl.col("UVT_SMMLV")))
                                                         .then((pl.lit(42.54) * pl.col("UVT_SMMLV")))
                                                         .otherwise((pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO") * pl.lit(0.017)))
                                                     )

                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "COMERCIAL")
                                                     .then(
                                                         pl
                                                         .when((pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO") * pl.lit(0.017)) > (pl.lit(35) * pl.col("UVT_SMMLV")))
                                                         .then((pl.lit(35) * pl.col("UVT_SMMLV")))
                                                         .otherwise((pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO") * pl.lit(0.017)))
                                                     )
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESPECIAL")
                                                     .then(pl.lit(0.017) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "OTROS")
                                                     .then(pl.lit(0.017) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "OFICIAL - PUBLICO")
                                                     .then(pl.lit(0.01) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESTRATO 4")
                                                     .then(pl.lit(0.01) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESTRATO 5")
                                                     .then(pl.lit(0.02) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESTRATO 6")
                                                     .then(pl.lit(0.02) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .otherwise(pl.lit(0.01) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO")).alias("TESCC"))


#low limit uvt
ft01_or573_2021_df = ft01_or573_2021_df.with_columns(pl
                                                     .when(pl.col("TESCC") < (pl.lit(0.05) * pl.col("UVT_SMMLV")))
                                                     .then((pl.lit(0.05) * pl.col("UVT_SMMLV")))
                                                     .otherwise(pl.col("TESCC"))
                                                     .alias("TESCC"))

display(ft01_or573_2021_df.head())

TIPO_DOCUMENTO,NUMERO_DOCUMENTO,DIGITO_VERIFICACION,PRIMER_NOMBRE_RAZON_SOCIAL,SEGUNDO_NOMBRE,PRIMER_APELLIDO,SEGUNDO_APELLIDO,NIU,NIC_CODIGO_ID,MEDIDOR,DIRECCION_ENVIO,CIUDAD_ENVIO,DIRECCION_SUMINISTRO,BARRIO_SUMINISTRO,CORREGIMIENTO_SUMINISTRO,MUNICIPIO_SUMINISTRO,MERCADO,SECTOR,DESTINO_ECONOMICO_PREDIO,ESTRATO,NUMERO_FACTURA,PERIODO_FACTURADO_INICIO,PERIODO_FACTURADO_FINAL,ANIO_VIGENCIA,PERIODO_MES,CICLO_FACTURACION,FECHA_VENCIMIENTO_FACTURA,CONSUMO_KWH,CONSUMO_KWH_SUBSIDIADO,CONSUMO_ENERGIA_SIN_SUBSIDIO,CONSUMO_ENERGIA_SUBSIDIADO,TARIFA_APLICADA_KWH,TARIFA_SUBSIDIADA_KWH,FICHA_CATASTRAL,CORREO_ELECTRONICO,OPERADOR,UVT_SMMLV,TESCC
str,i64,i64,str,str,str,str,i64,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,i64,str,i64,i32,f64
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,337832857,"""09-04-2022""","""10-05-2022""",2022,5,35,"""01-06-2022""",4840.0,0.0,3.5215e6,3.5215e6,727.59,727.59,1,null,165,38004,59865.79257
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,333629402,"""08-02-2022""","""09-03-2022""",2022,3,35,"""01-04-2022""",6920.0,0.0,4.7477e6,4.7477e6,686.08,686.08,1,null,165,38004,80710.26301
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686188,1017845,"""ELEN60151808""","""CL 18 128-233 …",76001,"""CL 18 128-233 …","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""COMERCIAL""",99,329523648,"""08-12-2021""","""06-01-2022""",2022,1,35,"""01-02-2022""",219.0,0.0,141626.14,141626.14,646.69,646.69,1,null,165,38004,2407.64438
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,329524124,"""08-12-2021""","""06-01-2022""",2022,1,35,"""01-02-2022""",4000.0,0.0,2586778.8,2586778.8,646.69,646.69,1,null,165,38004,43975.2396
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,342070010,"""10-06-2022""","""08-07-2022""",2022,7,35,"""29-07-2022""",4320.0,0.0,3391924.9,3391924.9,785.17,785.17,1,null,165,38004,57662.7233


In [69]:
#separate by year
ft01_or425_2016_df = ft01_or425_2016_df.filter((pl.col("ANIO_VIGENCIA") <= 2021))
#ft01_or425_2016_df.select(pl.count())

#set smmlv
ft01_or425_2016_df = ft01_or425_2016_df.with_columns(pl
                                                     .when(pl.col("ANIO_VIGENCIA") == 2017)
                                                     .then(737717)
                                                     .when(pl.col("ANIO_VIGENCIA") == 2018)
                                                     .then(781242)
                                                     .when(pl.col("ANIO_VIGENCIA") == 2019)
                                                     .then(828116)
                                                     .when(pl.col("ANIO_VIGENCIA") == 2020)
                                                     .then(877803)
                                                     .when(pl.col("ANIO_VIGENCIA") == 2021)
                                                     .then(908526)
                                                     .otherwise(908526).alias("UVT_SMMLV")) #smmlv -2021


#TESCC before 2021
ft01_or425_2016_df = ft01_or425_2016_df.with_columns(pl
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "INDUSTRIAL")
                                                     .then(
                                                         pl
                                                         .when(pl.col("CONSUMO_KWH") >= 300000)
                                                         .then(pl.lit(1.7) * pl.col("UVT_SMMLV"))
                                                         .otherwise(pl.lit(0.017) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     )
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "COMERCIAL")
                                                     .then(
                                                         pl
                                                         .when(pl.col("CONSUMO_KWH") >= 200000)
                                                         .then(pl.lit(1.4) * pl.col("UVT_SMMLV"))
                                                         .otherwise(pl.lit(0.017) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     )
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESPECIAL")
                                                     .then(pl.lit(0.017) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "OTROS")
                                                     .then(pl.lit(0.017) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "OFICIAL - PUBLICO")
                                                     .then(pl.lit(0.01) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESTRATO 4")
                                                     .then(pl.lit(0.01) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESTRATO 5")
                                                     .then(pl.lit(0.02) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .when(pl.col("DESTINO_ECONOMICO_PREDIO") == "ESTRATO 6")
                                                     .then(pl.lit(0.02) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO"))
                                                     .otherwise(pl.lit(0.01) * pl.col("CONSUMO_ENERGIA_SIN_SUBSIDIO")).alias("TESCC"))

display(ft01_or425_2016_df.head())

TIPO_DOCUMENTO,NUMERO_DOCUMENTO,DIGITO_VERIFICACION,PRIMER_NOMBRE_RAZON_SOCIAL,SEGUNDO_NOMBRE,PRIMER_APELLIDO,SEGUNDO_APELLIDO,NIU,NIC_CODIGO_ID,MEDIDOR,DIRECCION_ENVIO,CIUDAD_ENVIO,DIRECCION_SUMINISTRO,BARRIO_SUMINISTRO,CORREGIMIENTO_SUMINISTRO,MUNICIPIO_SUMINISTRO,MERCADO,SECTOR,DESTINO_ECONOMICO_PREDIO,ESTRATO,NUMERO_FACTURA,PERIODO_FACTURADO_INICIO,PERIODO_FACTURADO_FINAL,ANIO_VIGENCIA,PERIODO_MES,CICLO_FACTURACION,FECHA_VENCIMIENTO_FACTURA,CONSUMO_KWH,CONSUMO_KWH_SUBSIDIADO,CONSUMO_ENERGIA_SIN_SUBSIDIO,CONSUMO_ENERGIA_SUBSIDIADO,TARIFA_APLICADA_KWH,TARIFA_SUBSIDIADA_KWH,FICHA_CATASTRAL,CORREO_ELECTRONICO,OPERADOR,UVT_SMMLV,TESCC
str,i64,i64,str,str,str,str,i64,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,i64,str,i64,i32,f64
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,354013788,"""07-01-2022""","""07-02-2021""",2021,2,35,"""07-03-2022""",6880.0,0.0,4.5827e6,4.5827e6,666.1,666.1,1,null,165,908526,77906.54141
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,335672446,"""10-03-2022""","""08-04-2021""",2021,4,35,"""04-05-2022""",6360.0,0.0,4.5816e6,4.5816e6,720.38,720.38,1,null,165,908526,77887.73431


### union ft01

In [70]:
#union ft01
ft01_df = pl.concat([ft01_or425_2016_df, ft01_or573_2021_df])
display(ft01_df.head())

TIPO_DOCUMENTO,NUMERO_DOCUMENTO,DIGITO_VERIFICACION,PRIMER_NOMBRE_RAZON_SOCIAL,SEGUNDO_NOMBRE,PRIMER_APELLIDO,SEGUNDO_APELLIDO,NIU,NIC_CODIGO_ID,MEDIDOR,DIRECCION_ENVIO,CIUDAD_ENVIO,DIRECCION_SUMINISTRO,BARRIO_SUMINISTRO,CORREGIMIENTO_SUMINISTRO,MUNICIPIO_SUMINISTRO,MERCADO,SECTOR,DESTINO_ECONOMICO_PREDIO,ESTRATO,NUMERO_FACTURA,PERIODO_FACTURADO_INICIO,PERIODO_FACTURADO_FINAL,ANIO_VIGENCIA,PERIODO_MES,CICLO_FACTURACION,FECHA_VENCIMIENTO_FACTURA,CONSUMO_KWH,CONSUMO_KWH_SUBSIDIADO,CONSUMO_ENERGIA_SIN_SUBSIDIO,CONSUMO_ENERGIA_SUBSIDIADO,TARIFA_APLICADA_KWH,TARIFA_SUBSIDIADA_KWH,FICHA_CATASTRAL,CORREO_ELECTRONICO,OPERADOR,UVT_SMMLV,TESCC
str,i64,i64,str,str,str,str,i64,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,i64,str,i64,i32,f64
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,354013788,"""07-01-2022""","""07-02-2021""",2021,2,35,"""07-03-2022""",6880.0,0.0,4.5827e6,4.5827e6,666.1,666.1,1,null,165,908526,77906.54141
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,335672446,"""10-03-2022""","""08-04-2021""",2021,4,35,"""04-05-2022""",6360.0,0.0,4.5816e6,4.5816e6,720.38,720.38,1,null,165,908526,77887.73431
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,337832857,"""09-04-2022""","""10-05-2022""",2022,5,35,"""01-06-2022""",4840.0,0.0,3.5215e6,3.5215e6,727.59,727.59,1,null,165,38004,59865.79257
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,333629402,"""08-02-2022""","""09-03-2022""",2022,3,35,"""01-04-2022""",6920.0,0.0,4.7477e6,4.7477e6,686.08,686.08,1,null,165,38004,80710.26301
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686188,1017845,"""ELEN60151808""","""CL 18 128-233 …",76001,"""CL 18 128-233 …","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""COMERCIAL""",99,329523648,"""08-12-2021""","""06-01-2022""",2022,1,35,"""01-02-2022""",219.0,0.0,141626.14,141626.14,646.69,646.69,1,null,165,38004,2407.64438


In [71]:
# 
#display(ft01_df.select(
#    [
#        pl.format("{}", pl.col("CONSUMO_ENERGIA_SUBSIDIADO").round(2)),
#        ]
#    )
#       )

### ft03

In [72]:
#read parquet files ft03
ft03_df = pl.read_parquet("./files/input/remaining/ft03/*")

ft03_df = ft03_df.with_columns(pl.col("NUMERO_FACTURA").alias("NUMERO_FACTURA_FT03"))

display(ft03_df.head())

NUMERO_DOCUMENTO,DIGITO_VERIFICACION,NIU,NIC_CODIGO_ID,NUMERO_FACTURA,VALOR_FACTURADO_TASA_SEGURIDAD,VALOR_INTERESES,FECHA_LIMITE_PAGO,VALOR_RECAUDO,FECHA_RECAUDO,NUMERO_FACTURA_FT03
i64,str,i64,i64,i64,f64,f64,str,f64,str,i64
16623043,null,151314,93464,354013788,0.0,0.0,"""18-01-2023""",11.86,"""26-01-2023""",354013788
2557306,null,129355,87059,354009198,0.0,0.0,"""18-01-2023""",6183.52,"""26-01-2023""",354009198
31266414,null,127938,86753,354009047,1900.2,0.0,"""18-01-2023""",1900.2,"""13-01-2023""",354009047
16625223,null,128074,86782,354009049,3844.17,0.0,"""18-01-2023""",3844.17,"""18-01-2023""",354009049
34592014,null,127956,86757,354009050,3001.41,0.0,"""18-01-2023""",3001.41,"""18-01-2023""",354009050


In [73]:
# report
df_report = ft01_df.join(ft03_df, on="NUMERO_FACTURA")
display(df_report.head())

TIPO_DOCUMENTO,NUMERO_DOCUMENTO,DIGITO_VERIFICACION,PRIMER_NOMBRE_RAZON_SOCIAL,SEGUNDO_NOMBRE,PRIMER_APELLIDO,SEGUNDO_APELLIDO,NIU,NIC_CODIGO_ID,MEDIDOR,DIRECCION_ENVIO,CIUDAD_ENVIO,DIRECCION_SUMINISTRO,BARRIO_SUMINISTRO,CORREGIMIENTO_SUMINISTRO,MUNICIPIO_SUMINISTRO,MERCADO,SECTOR,DESTINO_ECONOMICO_PREDIO,ESTRATO,NUMERO_FACTURA,PERIODO_FACTURADO_INICIO,PERIODO_FACTURADO_FINAL,ANIO_VIGENCIA,PERIODO_MES,CICLO_FACTURACION,FECHA_VENCIMIENTO_FACTURA,CONSUMO_KWH,CONSUMO_KWH_SUBSIDIADO,CONSUMO_ENERGIA_SIN_SUBSIDIO,CONSUMO_ENERGIA_SUBSIDIADO,TARIFA_APLICADA_KWH,TARIFA_SUBSIDIADA_KWH,FICHA_CATASTRAL,CORREO_ELECTRONICO,OPERADOR,UVT_SMMLV,TESCC,NUMERO_DOCUMENTO_right,DIGITO_VERIFICACION_right,NIU_right,NIC_CODIGO_ID_right,VALOR_FACTURADO_TASA_SEGURIDAD,VALOR_INTERESES,FECHA_LIMITE_PAGO,VALOR_RECAUDO,FECHA_RECAUDO,NUMERO_FACTURA_FT03
str,i64,i64,str,str,str,str,i64,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,i64,str,i64,i32,f64,i64,str,i64,i64,f64,f64,str,f64,str,i64
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,354013788,"""07-01-2022""","""07-02-2021""",2021,2,35,"""07-03-2022""",6880.0,0.0,4.5827e6,4.5827e6,666.1,666.1,1,null,165,908526,77906.54141,16623043,null,151314,93464,0.0,0.0,"""18-01-2023""",11.86,"""26-01-2023""",354013788
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,354013788,"""07-01-2022""","""07-02-2021""",2021,2,35,"""07-03-2022""",6880.0,0.0,4.5827e6,4.5827e6,666.1,666.1,1,null,165,908526,77906.54141,16623043,null,151314,93464,1900.2,0.0,"""18-01-2023""",1888.34,"""06-01-2023""",354013788


In [74]:
#saldo cartera
df_report = df_report.with_columns((pl.col("TESCC") - pl.col("VALOR_RECAUDO")).alias('CARTERA FT01-FT03'))


In [75]:
display(df_report.head())

TIPO_DOCUMENTO,NUMERO_DOCUMENTO,DIGITO_VERIFICACION,PRIMER_NOMBRE_RAZON_SOCIAL,SEGUNDO_NOMBRE,PRIMER_APELLIDO,SEGUNDO_APELLIDO,NIU,NIC_CODIGO_ID,MEDIDOR,DIRECCION_ENVIO,CIUDAD_ENVIO,DIRECCION_SUMINISTRO,BARRIO_SUMINISTRO,CORREGIMIENTO_SUMINISTRO,MUNICIPIO_SUMINISTRO,MERCADO,SECTOR,DESTINO_ECONOMICO_PREDIO,ESTRATO,NUMERO_FACTURA,PERIODO_FACTURADO_INICIO,PERIODO_FACTURADO_FINAL,ANIO_VIGENCIA,PERIODO_MES,CICLO_FACTURACION,FECHA_VENCIMIENTO_FACTURA,CONSUMO_KWH,CONSUMO_KWH_SUBSIDIADO,CONSUMO_ENERGIA_SIN_SUBSIDIO,CONSUMO_ENERGIA_SUBSIDIADO,TARIFA_APLICADA_KWH,TARIFA_SUBSIDIADA_KWH,FICHA_CATASTRAL,CORREO_ELECTRONICO,OPERADOR,UVT_SMMLV,TESCC,NUMERO_DOCUMENTO_right,DIGITO_VERIFICACION_right,NIU_right,NIC_CODIGO_ID_right,VALOR_FACTURADO_TASA_SEGURIDAD,VALOR_INTERESES,FECHA_LIMITE_PAGO,VALOR_RECAUDO,FECHA_RECAUDO,NUMERO_FACTURA_FT03,CARTERA FT01-FT03
str,i64,i64,str,str,str,str,i64,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,i64,str,i64,i32,f64,i64,str,i64,i64,f64,f64,str,f64,str,i64,f64
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,354013788,"""07-01-2022""","""07-02-2021""",2021,2,35,"""07-03-2022""",6880.0,0.0,4.5827e6,4.5827e6,666.1,666.1,1,null,165,908526,77906.54141,16623043,null,151314,93464,0.0,0.0,"""18-01-2023""",11.86,"""26-01-2023""",354013788,77894.68141
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,354013788,"""07-01-2022""","""07-02-2021""",2021,2,35,"""07-03-2022""",6880.0,0.0,4.5827e6,4.5827e6,666.1,666.1,1,null,165,908526,77906.54141,16623043,null,151314,93464,1900.2,0.0,"""18-01-2023""",1888.34,"""06-01-2023""",354013788,76018.20141


In [76]:
# Getting rows in FT01 only if not present in FT03
df_exclude = ft01_df.join(ft03_df, on="NUMERO_FACTURA", how="left").filter(pl.all(pl.col('NUMERO_FACTURA_FT03').is_null()))

display(df_exclude)

TIPO_DOCUMENTO,NUMERO_DOCUMENTO,DIGITO_VERIFICACION,PRIMER_NOMBRE_RAZON_SOCIAL,SEGUNDO_NOMBRE,PRIMER_APELLIDO,SEGUNDO_APELLIDO,NIU,NIC_CODIGO_ID,MEDIDOR,DIRECCION_ENVIO,CIUDAD_ENVIO,DIRECCION_SUMINISTRO,BARRIO_SUMINISTRO,CORREGIMIENTO_SUMINISTRO,MUNICIPIO_SUMINISTRO,MERCADO,SECTOR,DESTINO_ECONOMICO_PREDIO,ESTRATO,NUMERO_FACTURA,PERIODO_FACTURADO_INICIO,PERIODO_FACTURADO_FINAL,ANIO_VIGENCIA,PERIODO_MES,CICLO_FACTURACION,FECHA_VENCIMIENTO_FACTURA,CONSUMO_KWH,CONSUMO_KWH_SUBSIDIADO,CONSUMO_ENERGIA_SIN_SUBSIDIO,CONSUMO_ENERGIA_SUBSIDIADO,TARIFA_APLICADA_KWH,TARIFA_SUBSIDIADA_KWH,FICHA_CATASTRAL,CORREO_ELECTRONICO,OPERADOR,UVT_SMMLV,TESCC,NUMERO_DOCUMENTO_right,DIGITO_VERIFICACION_right,NIU_right,NIC_CODIGO_ID_right,VALOR_FACTURADO_TASA_SEGURIDAD,VALOR_INTERESES,FECHA_LIMITE_PAGO,VALOR_RECAUDO,FECHA_RECAUDO,NUMERO_FACTURA_FT03
str,i64,i64,str,str,str,str,i64,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,i64,str,i64,i32,f64,i64,str,i64,i64,f64,f64,str,f64,str,i64
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,335672446,"""10-03-2022""","""08-04-2021""",2021,4,35,"""04-05-2022""",6360.0,0.0,4.5816e6,4.5816e6,720.38,720.38,1,null,165,908526,77887.73431,null,null,null,null,null,null,null,null,null,null
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,337832857,"""09-04-2022""","""10-05-2022""",2022,5,35,"""01-06-2022""",4840.0,0.0,3.5215e6,3.5215e6,727.59,727.59,1,null,165,38004,59865.79257,null,null,null,null,null,null,null,null,null,null
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,333629402,"""08-02-2022""","""09-03-2022""",2022,3,35,"""01-04-2022""",6920.0,0.0,4.7477e6,4.7477e6,686.08,686.08,1,null,165,38004,80710.26301,null,null,null,null,null,null,null,null,null,null
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686188,1017845,"""ELEN60151808""","""CL 18 128-233 …",76001,"""CL 18 128-233 …","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""COMERCIAL""",99,329523648,"""08-12-2021""","""06-01-2022""",2022,1,35,"""01-02-2022""",219.0,0.0,141626.14,141626.14,646.69,646.69,1,null,165,38004,2407.64438,null,null,null,null,null,null,null,null,null,null
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,329524124,"""08-12-2021""","""06-01-2022""",2022,1,35,"""01-02-2022""",4000.0,0.0,2586778.8,2586778.8,646.69,646.69,1,null,165,38004,43975.2396,null,null,null,null,null,null,null,null,null,null
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,342070010,"""10-06-2022""","""08-07-2022""",2022,7,35,"""29-07-2022""",4320.0,0.0,3391924.9,3391924.9,785.17,785.17,1,null,165,38004,57662.7233,null,null,null,null,null,null,null,null,null,null
"""NIT""",890319982,8,"""COLEGIO COOPER…",null,null,null,1686181,1017843,"""U41A68052854""","""CL 18 128-233""",76001,"""CL 18 128-233""","""LOS NARANJOS I…",null,76001,"""R""","""URBANO""","""ESPECIAL""",99,339986132,"""11-05-2022""","""09-06-2022""",2022,6,35,"""06-07-2022""",5360.0,0.0,4.0949e6,4.0949e6,763.97,763.97,1,null,165,38004,69612.52718,null,null,null,null,null,null,null,null,null,null


# SAVE

In [77]:
# filenames
letters_var = string.ascii_letters
success_var =  ( ''.join(random.choice(letters_var) for i in range(15)) )
error_var = ( ''.join(random.choice(letters_var) for i in range(15)) )

In [78]:
#save report
#to csv
df_report.write_csv("./files/output/completed/found_num_fact/report_" + success_var + ".csv")

In [79]:
#save not found records
#to csv
df_exclude.write_csv("./files/output/completed/not_found/not_found_" + error_var + ".csv")

### Pruebas - borrar a partir de aquí

In [ ]:

#pyarrow.dataset.write_dataset(
#    ft01_or425_2016_df.to_arrow(),
#    "./files/input/remaining",
#    format="parquet",
#    partitioning=["ANIO_VIGENCIA"],
#    partitioning_flavor="hive",
#    existing_data_behavior="delete_matching",
#)

In [ ]:
#to scv

df_report.write_csv("./files/output/polars_data.csv")

In [ ]:
#df1 = pl.read_parquet("./files/input/remaining/*")

#display(df1.head())

# BORRAR

In [ ]:
#parquet
#ft01_or425_2016_df = pl.read_parquet("./files/input/remaining/ft01/*")


# ft01 input
file_url_ft01 = (
    "files/input/test/FT01_EMCALI_-_PARA_FACTURAR_ENERO_A_JULIO_2022_COLEGIO_COOMEVA.txt"
)
ft01_or425_2016_df = pl.read_csv(file_url_ft01, separator= "|", has_header= False)

# ft01 rename
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_1": "TIPO_DOCUMENTO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_2": "NUMERO_DOCUMENTO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_3": "DIGITO_VERIFICACION"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_4": "PRIMER_NOMBRE_RAZON_SOCIAL"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_5": "SEGUNDO_NOMBRE"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_6": "PRIMER_APELLIDO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_7": "SEGUNDO_APELLIDO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_8": "NIU"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_9": "NIC_CODIGO_ID"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_10": "MEDIDOR"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_11": "DIRECCION_ENVIO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_12": "CIUDAD_ENVIO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_13": "DIRECCION_SUMINISTRO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_14": "BARRIO_SUMINISTRO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_15": "CORREGIMIENTO_SUMINISTRO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_16": "MUNICIPIO_SUMINISTRO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_17": "MERCADO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_18": "SECTOR"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_19": "DESTINO_ECONOMICO_PREDIO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_20": "ESTRATO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_21": "NUMERO_FACTURA"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_22": "PERIODO_FACTURADO_INICIO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_23": "PERIODO_FACTURADO_FINAL"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_24": "ANIO_VIGENCIA"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_25": "PERIODO_MES"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_26": "CICLO_FACTURACION"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_27": "FECHA_VENCIMIENTO_FACTURA"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_28": "CONSUMO_KWH"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_29": "CONSUMO_KWH_SUBSIDIADO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_30": "CONSUMO_ENERGIA_SIN_SUBSIDIO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_31": "CONSUMO_ENERGIA_SUBSIDIADO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_32": "TARIFA_APLICADA_KWH"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_33": "TARIFA_SUBSIDIADA_KWH"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_34": "FICHA_CATASTRAL"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_35": "CORREO_ELECTRONICO"})
ft01_or425_2016_df = ft01_or425_2016_df.rename({"column_36": "OPERADOR"})

In [ ]:
display(ft01_or425_2016_df.head())

In [ ]:
pyarrow.dataset.write_dataset(
    ft01_or425_2016_df.to_arrow(),
    "./files/input/test",
    format="parquet",
    partitioning_flavor="hive",
    existing_data_behavior="delete_matching",
)

In [ ]:
#parquet
#ft03_df = pl.read_parquet("./files/input/remaining/ft03/*")

# ft03 input
file_url_ft03 = (
    "files/input/test/20230214220009_890399003_2023_01_FT03_03.csv"
)
ft03_df = pl.read_csv(file_url_ft03, separator= "|", has_header= False)

# ft03 rename
ft03_df = ft03_df.rename({"column_1": "NUMERO_DOCUMENTO"})
ft03_df = ft03_df.rename({"column_2": "DIGITO_VERIFICACION"})
ft03_df = ft03_df.rename({"column_3": "NIU"})
ft03_df = ft03_df.rename({"column_4": "NIC_CODIGO_ID"})
ft03_df = ft03_df.rename({"column_5": "NUMERO_FACTURA"})
ft03_df = ft03_df.rename({"column_6": "VALOR_FACTURADO_TASA_SEGURIDAD"})
ft03_df = ft03_df.rename({"column_7": "VALOR_INTERESES"})
ft03_df = ft03_df.rename({"column_8": "FECHA_LIMITE_PAGO"})
ft03_df = ft03_df.rename({"column_9": "VALOR_RECAUDO"})
ft03_df = ft03_df.rename({"column_10": "FECHA_RECAUDO"})

In [ ]:
display(ft03_df.head())

In [ ]:
pyarrow.dataset.write_dataset(
    ft03_df.to_arrow(),
    "./files/input/remaining/ft03",
    format="parquet",
    partitioning_flavor="hive",
    existing_data_behavior="delete_matching",
)

In [ ]:
#parquet
ft01_dfff = pl.read_parquet("./files/input/remaining/ft01/*")

In [ ]:
display(ft01_dfff.head())